In [1]:
import pandas as pd
import numpy as np

## Fetch products

In [2]:
product_df = pd.read_csv("dataset/product.csv", sep='\t')
product_df.head()

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


In [3]:
## Assemble the product text use our product description as that text unless there is no description in which case we will use the product name.

In [4]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42994 entries, 0 to 42993
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           42994 non-null  int64  
 1   product_name         42994 non-null  object 
 2   product_class        40142 non-null  object 
 3   category hierarchy   41438 non-null  object 
 4   product_description  36986 non-null  object 
 5   product_features     42994 non-null  object 
 6   rating_count         33542 non-null  float64
 7   average_rating       33542 non-null  float64
 8   review_count         33542 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 3.0+ MB


In [5]:
product_df['product_text'] = np.where(pd.notna(product_df['product_description']), 
                                      product_df['product_description'], 
                                      product_df['product_name'])

In [6]:
product_df.iloc[42990]

product_id                                                         42990
product_name                       emmeline 5 piece breakfast dining set
product_class                                          Dining Table Sets
category hierarchy     Furniture / Kitchen & Dining Furniture / Dinin...
product_description                                                  NaN
product_features       basematerialdetails : steel| : gray wood|ofhar...
rating_count                                                      1314.0
average_rating                                                       4.5
review_count                                                       864.0
product_text                       emmeline 5 piece breakfast dining set
Name: 42990, dtype: object

In [7]:
from llama_index.core import Document

documents = [
    Document(
        text=row['product_text'],
        metadata={
            'product_id': row['product_id'],
            'product_name': row['product_name'],
        }
    )
    for _, row in product_df.iterrows()
]

In [8]:
print(documents[0])

Doc ID: 9319c511-4357-4820-8195-33135cc2f970
Text: good , deep sleep can be quite difficult to have in this busy
age . fortunately , there ’ s an antidote to such a problem : a nice ,
quality bed frame like the acacia kaylin . solidly constructed from
acacia wood , this bed frame will stand the test of time and is fit to
rest your shoulders on for years and years . its sleek , natural wood
grain...


### Extract Nodes

In [9]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

In [10]:
# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [11]:
# Extract nodes from documents
nodes = node_parser.get_nodes_from_documents(documents)

In [12]:
print(f"Text: \n{nodes[0].text}")
print("------------------")
print(f"Window: \n{nodes[0].metadata['window']}")

Text: 
good , deep sleep can be quite difficult to have in this busy age . 
------------------
Window: 
good , deep sleep can be quite difficult to have in this busy age .  fortunately , there ’ s an antidote to such a problem : a nice , quality bed frame like the acacia kaylin .  solidly constructed from acacia wood , this bed frame will stand the test of time and is fit to rest your shoulders on for years and years .  its sleek , natural wood grain appearance provides a pleasant aesthetic to adorn any bedroom , acting both as a decorative piece as well as a place to give comfort after a hard day of work . 


In [13]:
len(nodes)

164163

### Convert Product Info into Embeddings

In [14]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
# openai.api_key = os.getenv('OPENAI_API_KEY')

### Creating a Weaviate Client

In [15]:
import weaviate

In [16]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [17]:
client = weaviate.Client(
    url=os.getenv("WEAVIATE_URL"), # Replace with your Weaviate Cloud URL
    auth_client_secret=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_API_KEY")),  # Replace w/ your Weaviate instance API key
    additional_headers={"X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],}
)

/Users/linghuang/miniconda3/envs/llama-index4/lib/python3.8/site-packages/weaviate/warnings.py:121: DeprecationWarning: Dep005: You are using weaviate-client version 3.26.2. The latest version is 4.6.3.
            Please consider upgrading to the latest version. See https://weaviate.io/developers/weaviate/client-libraries/python for details.
  warnings.warn(


In [18]:
client.is_ready()

True

### Next, you will build a `VectorStoreIndex` from the Weaviate client to store your data in and interact with.

In [19]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore

In [20]:
# construct vector store
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="LlamaIndex"
)

In [21]:
# Set up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [22]:
# If an index with the same index name already exists within Weaviate, delete it
# if client.schema.exists("LlamaIndex"):
#     client.schema.delete_class("LlamaIndex")

# index = VectorStoreIndex(
#     nodes,
#     storage_context = storage_context,
# )

In [23]:
import json

index_name="LlamaIndex"
response = client.schema.get(index_name)
# print(json.dumps(response, indent=2))

### Loading the index

In [24]:
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="LlamaIndex"
)

index = VectorStoreIndex.from_vector_store(vector_store)

### Query Index

In [25]:
query_engine = index.as_query_engine(                                                                           
    vector_store_query_mode="hybrid", similarity_top_k=2
)
response = query_engine.query(                                                                                     
    "salon chair",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [26]:
print(response.metadata)

{'e05f0fc2-15d3-40e1-a17f-d4ba60f6199b': {'window': 'offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon', 'original_text': 'to showcase the uniqueness of each salon', 'product_id': 7465, 'product_name': 'hair salon chair'}, 'ed7967fc-f7da-47c1-86c5-445027ee1fcd': {'window': 'office work chair beauty salon chair white', 'original_text': 'office work chair beauty salon chair white', 'product_id': 33690, 'product_name': 'office work chair beauty salon chair white'}}


In [27]:
import json
print(json.dumps(response.metadata, indent=2))

{
  "e05f0fc2-15d3-40e1-a17f-d4ba60f6199b": {
    "window": "offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon",
    "original_text": "to showcase the uniqueness of each salon",
    "product_id": 7465,
    "product_name": "hair salon chair"
  },
  "ed7967fc-f7da-47c1-86c5-445027ee1fcd": {
    "window": "office work chair beauty salon chair white",
    "original_text": "office work chair beauty salon chair white",
    "product_id": 33690,
    "product_name": "office work chair beauty salon chair white"
  }
}


In [28]:
# get manually labeled groundtruth lables
label_df = pd.read_csv("dataset/label.csv", sep='\t')

filtered_df = label_df[(label_df['query_id'] == 0) & (label_df['product_id'].isin([7465, 33690]))]
result_dict = filtered_df[['query_id', 'product_id', 'label']]
print(result_dict)

     query_id  product_id    label
80          0        7465    Exact
109         0       33690  Partial


### Query With MetadataReplacementPostProcessor

In [32]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

In [33]:
query_engine = index.as_query_engine(
    vector_store_query_mode="hybrid", 
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [34]:
window_response = query_engine.query(
    "salon chair"
)
print(window_response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The product mentioned in the context is a hair salon chair.


In [35]:
print(json.dumps(window_response.metadata, indent=2))

{
  "e05f0fc2-15d3-40e1-a17f-d4ba60f6199b": {
    "window": "offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon",
    "original_text": "to showcase the uniqueness of each salon",
    "product_id": 7465,
    "product_name": "hair salon chair"
  },
  "ed7967fc-f7da-47c1-86c5-445027ee1fcd": {
    "window": "office work chair beauty salon chair white",
    "original_text": "office work chair beauty salon chair white",
    "product_id": 33690,
    "product_name": "office work chair beauty salon chair white"
  }
}


In [36]:
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon
------------------
Original Sentence: to showcase the uniqueness of each salon


In [37]:
for source_node in window_response.source_nodes:
    print(source_node.node.metadata["original_text"])
    print("--------")

to showcase the uniqueness of each salon
--------
office work chair beauty salon chair white
--------


### LLM Reranker

In [71]:
import nest_asyncio

nest_asyncio.apply()

In [72]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [73]:
from llama_index.core.postprocessor import LLMRerank

query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=2,
        )
    ],
    response_mode="tree_summarize",
)
response = query_engine.query(
    "salon chair",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST h

In [74]:
print(json.dumps(response.metadata, indent=2))

{
  "e05f0fc2-15d3-40e1-a17f-d4ba60f6199b": {
    "window": "offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon",
    "original_text": "to showcase the uniqueness of each salon",
    "product_id": 7465,
    "product_name": "hair salon chair"
  },
  "ed7967fc-f7da-47c1-86c5-445027ee1fcd": {
    "window": "office work chair beauty salon chair white",
    "original_text": "office work chair beauty salon chair white",
    "product_id": 33690,
    "product_name": "office work chair beauty salon chair white"
  }
}


In [75]:
filtered_df = label_df[(label_df['query_id'] == 0) & (label_df['product_id'].isin([7465, 15612]))]
result_dict = filtered_df[['query_id', 'product_id', 'label']]
print(result_dict)

     query_id  product_id  label
80          0        7465  Exact
101         0       15612  Exact


In [76]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from llama_index.llms.openai import OpenAI

def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=2, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        reranker = LLMRerank(
            choice_batch_size=5,
            top_n=reranker_top_n,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )
    return retrieved_nodes

In [77]:
new_nodes = get_retrieved_nodes(
     "salon chair",
    vector_top_k=2,
    with_reranker=False,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [78]:
for node in new_nodes:
    # print(node)
    print(f"Score: {node.score:.2f} - {node.text[:200]}...")

Score: 0.90 - to showcase the uniqueness of each salon...
Score: 0.90 - it features a curved backrest design , an adjustable lift seat , and a five-star base with casters for easy mobility ....


In [79]:
new_nodes = get_retrieved_nodes(
    "salon chair",
    reranker_top_n=2,
    with_reranker=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [80]:
for node in new_nodes:
    # print(node)
    print(f"Score: {node.score:.2f} - {node.text[:200]}...")

Score: 10.00 - to showcase the uniqueness of each salon...
Score: 10.00 - office work chair beauty salon chair white...


### Query-Rewriting

In [95]:
import nest_asyncio

nest_asyncio.apply()

In [96]:
QUERY_GEN_PROMPT = (
    "You are a helpful assistant that generates multiple search queries based on a "
    "single input query. Generate {num_queries} search queries, one on each line, "
    "related to the following input query:\n"
    "Query: {query}\n"
    "Queries:\n"
)

In [97]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [index.as_retriever()],
    similarity_top_k=2,
    num_queries=5,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
    query_gen_prompt=QUERY_GEN_PROMPT,  
)

In [100]:
nodes_with_scores = retriever.retrieve("salon chair")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Generated queries:
1. Best salon chairs for professional use
2. Affordable salon chairs for sale
3. How to clean and maintain salon chairs
4. Different types of salon chairs and their uses
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: 

In [102]:
for node in nodes_with_scores:
    print(node)
    # print(f"Score: {node.score:.2f} - {node.text[:00]}...")

Node ID: 814c82a2-d82b-4214-be7b-dbdc8113fcf4
Text: salon chairs are a wonderful avenue for hairstylists and barbers
to increase their revenue - have a comfortable chair for customers to
enjoy during their visit .
Score:  0.050

Node ID: e05f0fc2-15d3-40e1-a17f-d4ba60f6199b
Text: to showcase the uniqueness of each salon
Score:  0.049



In [103]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever)

In [111]:
response = query_engine.query("salon chair")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Generated queries:
1. Best salon chairs for professionals
2. Affordable salon chairs for sale
3. How to clean and maintain salon chairs
4. Different types of salon chairs
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.o

In [112]:
print("Answer: {}".format(response))

Answer: A salon chair is a piece of furniture used in hair salons and barber shops. It is designed to provide comfort to customers during their visit. There are different types of salon chairs, such as a basic hair salon chair and a barberpub salon massage chair. These chairs not only serve a functional purpose but also contribute to the unique aesthetic of each salon. They can also help hairstylists and barbers increase their revenue.


In [113]:
print(json.dumps(response.metadata, indent=2))

{
  "e05f0fc2-15d3-40e1-a17f-d4ba60f6199b": {
    "window": "offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon",
    "original_text": "to showcase the uniqueness of each salon",
    "product_id": 7465,
    "product_name": "hair salon chair"
  },
  "814c82a2-d82b-4214-be7b-dbdc8113fcf4": {
    "window": "salon chairs are a wonderful avenue for hairstylists and barbers to increase their revenue - have a comfortable chair for customers to enjoy during their visit .",
    "original_text": "salon chairs are a wonderful avenue for hairstylists and barbers to increase their revenue - have a comfortable chair for customers to enjoy during their visit .",
    "product_id": 25431,
    "product_name": "barberpub salon massage chair"
  }
}


In [114]:
filtered_df = label_df[(label_df['query_id'] == 0) & (label_df['product_id'].isin([7465, 25431]))]
result_dict = filtered_df[['query_id', 'product_id', 'label']]
print(result_dict)

    query_id  product_id  label
29         0       25431  Exact
80         0        7465  Exact


### HyDE Query Transform 

In [115]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine

In [116]:
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=2,
        )
    ],
    response_mode="tree_summarize",
)

In [117]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query("salon chair")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1

In [123]:
print(response)

A salon chair is a piece of furniture used in salons by hairstylists and barbers. It provides a comfortable place for customers during their visit. Two examples of salon chairs are the Barberpub Salon Massage Chair and the Barberpub Hydraulic Reclining Massage Chair. These chairs can also help increase revenue for hairstylists and barbers.


In [119]:
print(json.dumps(response.metadata, indent=2))

{
  "814c82a2-d82b-4214-be7b-dbdc8113fcf4": {
    "window": "salon chairs are a wonderful avenue for hairstylists and barbers to increase their revenue - have a comfortable chair for customers to enjoy during their visit .",
    "original_text": "salon chairs are a wonderful avenue for hairstylists and barbers to increase their revenue - have a comfortable chair for customers to enjoy during their visit .",
    "product_id": 25431,
    "product_name": "barberpub salon massage chair"
  },
  "0465ebc3-6952-4822-968a-b644d0beff34": {
    "window": "salon chairs are a wonderful avenue for hairstylists and barbers to increase their revenue - have a comfortable chair for customers to enjoy during their visit .",
    "original_text": "salon chairs are a wonderful avenue for hairstylists and barbers to increase their revenue - have a comfortable chair for customers to enjoy during their visit .",
    "product_id": 25433,
    "product_name": "barberpub hydraulic reclining massage chair"
  }
}


In [122]:
filtered_df = label_df[(label_df['query_id'] == 0) & (label_df['product_id'].isin([25431, 25433]))]
result_dict = filtered_df[['query_id', 'product_id', 'label']]
print(result_dict)

    query_id  product_id  label
27         0       25433  Exact
29         0       25431  Exact


In [87]:
for node in new_nodes:
    print(node)
    # print(f"Score: {node.score:.2f} - {node.text[:200]}...")

Node ID: 7dbfbe32-6d80-4fc1-bf8c-1ec24f5a51b8
Text: the hydraulic salon chairs are made with a heavy-duty steel
frame , along with a premium chrome round base , sturdy arms , durable
hydraulic pump , and thick high-density foam seat and backrest .
Score:  10.000

Node ID: f85ed77c-27dc-4adb-bdcb-29782328ba19
Text: the hydraulic salon chairs are made with a heavy-duty steel
frame , along with a premium chrome round base , sturdy arms , durable
hydraulic pump , and thick high-density foam seat and backrest .
Score:  10.000



### Generator-Enhanced Retrieval  "Active Retrieval Augmented Generation"

In [81]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4", temperature=0)
Settings.chunk_size = 512

In [82]:
index_query_engine = index.as_query_engine(similarity_top_k=2)

In [83]:
from llama_index.core.query_engine import FLAREInstructQueryEngine

query_engine = FLAREInstructQueryEngine(
    query_engine=index_query_engine,
    max_iterations=2,
    verbose=True,
)

In [84]:
response = query_engine.query(
    "salon chair",
)

Query: salon chair
Current response: 
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Lookahead response: A salon chair is a type of furniture specifically designed for use in a beauty salon or barbershop, often featuring adjustable height and reclining capabilities for the comfort of the client and ease of the stylist. It is typically made of [Search(What materials are salon chairs typically made of?)]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INF

In [85]:
print(response)

A salon chair is a type of furniture specifically designed for use in a beauty salon or barbershop, often featuring adjustable height and reclining capabilities for the comfort of the client and ease of the stylist. It is typically made of a heavy-duty steel frame, a premium chrome round base, sturdy arms, a durable hydraulic pump, and a thick high-density foam seat and backrest. The upholstery can vary, but it is often made of faux leather for durability and easy cleaning.


In [86]:
new_nodes = get_retrieved_nodes(
    response.response,
    reranker_top_n=2,
    with_reranker=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [88]:
base_query_engine = index.as_query_engine()
response = base_query_engine.query("salon chair")
print(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The salon chair, with product id 7465, is designed to highlight the uniqueness of each salon. Another chair, the bar salon task chair with product id 24010, features a curved backrest design, an adjustable lift seat, and a five-star base with casters for easy mobility.


In [89]:
print(json.dumps(response.metadata, indent=2))

{
  "e05f0fc2-15d3-40e1-a17f-d4ba60f6199b": {
    "window": "offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon",
    "original_text": "to showcase the uniqueness of each salon",
    "product_id": 7465,
    "product_name": "hair salon chair"
  },
  "2c2fa1b0-e694-4eb6-a734-99f06ecb18b2": {
    "window": "the office chair is ideal for any casual or professional working area .  it provides comfortable support with a cozy and convenient design .  carefully crafted .  it features a curved backrest design , an adjustable lift seat , and a five-star base with casters for easy mobility .",
    "original_text": "it features a curved backrest design , an adjustable lift seat , and a five-star base with casters for easy mobility .",
    "product_id": 24010,
    "product_name": "bar salon task chair"
  }
}


In [91]:
filtered_df = label_df[(label_df['query_id'] == 0) & (label_df['product_id'].isin([7465, 24010]))]
result_dict = filtered_df[['query_id', 'product_id', 'label']]
print(result_dict)

    query_id  product_id    label
23         0       24010  Partial
80         0        7465    Exact


### Retry Query Engine

In [92]:
from llama_index.core.query_engine import RetryQueryEngine
from llama_index.core.evaluation import RelevancyEvaluator

query_response_evaluator = RelevancyEvaluator()
retry_query_engine = RetryQueryEngine(
    base_query_engine, query_response_evaluator
)
retry_response = retry_query_engine.query("salon chair")
print(retry_response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
The salon chair, with the produ

In [93]:
print(json.dumps(response.metadata, indent=2))

{
  "e05f0fc2-15d3-40e1-a17f-d4ba60f6199b": {
    "window": "offers a wide selection of professional salon products including styling chair , salon hairdryer , salon equipment , etc .  to showcase the uniqueness of each salon",
    "original_text": "to showcase the uniqueness of each salon",
    "product_id": 7465,
    "product_name": "hair salon chair"
  },
  "2c2fa1b0-e694-4eb6-a734-99f06ecb18b2": {
    "window": "the office chair is ideal for any casual or professional working area .  it provides comfortable support with a cozy and convenient design .  carefully crafted .  it features a curved backrest design , an adjustable lift seat , and a five-star base with casters for easy mobility .",
    "original_text": "it features a curved backrest design , an adjustable lift seat , and a five-star base with casters for easy mobility .",
    "product_id": 24010,
    "product_name": "bar salon task chair"
  }
}
